In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# YoloV5 Custom Training

In [7]:
%cd "/content"

/content


In [ ]:
#!unzip "/content/drive/MyDrive/zind_maps/ball_&_bat_tracking_hiring_challenge-dataset.zip"

In [ ]:
!unzip "/content/drive/MyDrive/zind_maps/cricket.zip"

## Clone the Github Repo

In [12]:
!git clone https://github.com/ultralytics/yolov5 
%cd yolov5
!git reset --hard 68211f72c99915a15855f7b99bf5d93f5631330f

Cloning into 'yolov5'...
remote: Enumerating objects: 8491, done.
remote: Counting objects: 100% (205/205), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 8491 (delta 104), reused 137 (delta 66), pack-reused 8286
Receiving objects: 100% (8491/8491), 9.61 MiB | 8.72 MiB/s, done.
Resolving deltas: 100% (5846/5846), done.
/content/yolov5
HEAD is now at 68211f7 FROM nvcr.io/nvidia/pytorch:20.10-py3 (#1553)


## Installing requirements

In [13]:
!pip install -qr requirements.txt
  # install dependencies (ignore errors)
!pip install torch==1.7.1



import torch

from IPython.display import Image, clear_output  # to display images
from utils.google_utils import gdrive_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

     |████████████████████████████████| 636 kB 10.0 MB/s 
     |████████████████████████████████| 776.8 MB 18 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu102 requires torch==1.9.0, but you have torch 1.7.1 which is incompatible.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.7.1 which is incompatible.
Setup complete. Using torch 1.7.1 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


In [14]:
!pip install torchvision==0.8.2

     |████████████████████████████████| 12.8 MB 24 kB/s 
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.10.0+cu102
    Uninstalling torchvision-0.10.0+cu102:
      Successfully uninstalled torchvision-0.10.0+cu102


## Checking the number of classes in our dataset

In [ ]:
#create an configuration file data.yaml

In [9]:
%cat /content/cricket/data.yaml

train: /content/cricket/images/train
val: /content/cricket/images/validation
nc: 2
names: ['ball','bat']

## # Defining number of classes

In [10]:
import yaml
with open("/content/cricket/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [11]:
num_classes

'2'

## Model configuration for YoloV5

In [15]:
%cat /content/yolov5/models/yolov5s.yaml

# parameters
nc: 80  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-

## Customize iPython writefile so we can write variables


In [16]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

## Model Configuration for our Model

In [ ]:
num_classes

'2'

In [17]:
%%writetemplate /content/yolov5/models/custom5_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

## Doing Changes in train.py for training

## Let's Start Training

In [ ]:
pwd

'/content/yolov5'

In [18]:
# train yolov5s on custom data for 100 epochs
# time its performance
%%time
#%cd /content/drive/MyDrive/YOLOV5_CUSTOM/yolov5
!python train.py --img 416 --batch 16 --epochs 500 --data '/content/cricket/data.yaml' --cfg /content/yolov5/models/custom5_yolov5s.yaml --weights '' --name yolov5s_cricket  --cache

Using torch 1.7.1 CUDA:0 (Tesla T4, 15109MB)

Namespace(adam=False, batch_size=16, bucket='', cache_images=True, cfg='/content/yolov5/models/custom5_yolov5s.yaml', data='/content/cricket/data.yaml', device='', epochs=500, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[416, 416], local_rank=-1, log_imgs=16, multi_scale=False, name='yolov5s_cricket', noautoanchor=False, nosave=False, notest=False, project='runs/train', rect=False, resume=False, save_dir='runs/train/yolov5s_cricket', single_cls=False, sync_bn=False, total_batch_size=16, weights='', workers=8, world_size=1)
Start Tensorboard with "tensorboard --logdir runs/train", view at http://localhost:6006/
2021-07-28 04:30:08.546744: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Hyperparameters {'lr0': 0.01, 'lrf': 0.2, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

## Inferencing with our custom Trained Model

In [19]:
%cd "/content/drive/MyDrive/yolov5"
!python detect.py --weights /content/yolov5/runs/train/yolov5s_cricket/weights/best.pt --img 416 --conf 0.4 --source /content/cricket/images/validation

[Errno 2] No such file or directory: '/content/drive/MyDrive/yolov5'
/content/yolov5
Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.4, device='', exist_ok=False, img_size=416, iou_thres=0.45, name='exp', project='runs/detect', save_conf=False, save_txt=False, source='/content/cricket/images/validation', update=False, view_img=False, weights=['/content/yolov5/runs/train/yolov5s_cricket/weights/best.pt'])
Using torch 1.7.1 CUDA:0 (Tesla T4, 15109MB)

Fusing layers... 
Model Summary: 232 layers, 7249215 parameters, 0 gradients
image 1/83 /content/cricket/images/validation/U11_0_8.png: 416x416 1 bats, Done. (0.010s)
image 2/83 /content/cricket/images/validation/U11_11_22.png: 416x416 1 balls, 1 bats, Done. (0.010s)
image 3/83 /content/cricket/images/validation/U11_11_25.png: 416x416 1 balls, 1 bats, Done. (0.012s)
image 4/83 /content/cricket/images/validation/U11_12_17.png: 416x416 1 balls, 1 bats, Done. (0.009s)
image 5/83 /content/cricket/images/validation/U11_13